In [61]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import time

In [2]:
data_og = pd.read_csv(f"../data/Options_data_2023.csv", index_col = 0)
data = data_og.copy()

In [ ]:
data["Time"] = pd.to_datetime(data["Time"], format = "%H:%M:%S").dt.time
data["Date"] = pd.to_datetime(data["Date"], format = "%Y-%m-%d").dt.normalize()

In [ ]:
data['Strike'] = data['Ticker'].str.extract(r'(\d+)').astype("float")
data["UnderlyingClose"] = data.groupby(["Date", "Time"])['Strike'].transform("median")

# Data Exploration

In [5]:
tickers = data["Ticker"].unique()
print("No. of tickers:" ,len(tickers))

No. of tickers: 322


In [6]:
data

,Date,Ticker,Time,Open,High,Low,Close,Call/Put,Timestamp,Strike,UnderlyingClose
0,2023-01-02,BANKNIFTY37000PE,09:15:59,3.40,3.40,2.85,3.00,PE,2023-01-02 09:15:59,37000.0,43000.0
1,2023-01-02,BANKNIFTY43800CE,09:15:59,79.85,104.25,73.90,91.20,CE,2023-01-02 09:15:59,43800.0,43000.0
2,2023-01-02,BANKNIFTY43900PE,09:15:59,867.95,883.15,826.75,826.75,PE,2023-01-02 09:15:59,43900.0,43000.0
3,2023-01-02,BANKNIFTY44000PE,09:15:59,957.25,981.05,911.00,914.70,PE,2023-01-02 09:15:59,44000.0,43000.0
4,2023-01-02,BANKNIFTY44200CE,09:15:59,37.90,42.95,27.35,37.10,CE,2023-01-02 09:15:59,44200.0,43000.0
...,...,...,...,...,...,...,...,...,...,...,...
10266676,2024-01-02,BANKNIFTY47000CE,15:29:59,615.00,615.95,612.15,614.95,CE,2024-01-02 15:29:59,47000.0,46900.0
10266677,2024-01-02,BANKNIFTY47000PE,15:29:59,11.95,12.45,11.65,12.25,PE,2024-01-02 15:29:59,47000.0,46900.0
10266678,2024-01-02,BANKNIFTY47500PE,15:29:59,106.45,109.50,105.10,107.65,PE,2024-01-02 15:29:59,47500.0,46900.0
10266679,2024-01-02,BANKNIFTY48000CE,15:29:59,44.70,45.00,43.00,44.60,CE,2024-01-02 15:29:59,48000.0,46900.0


In [7]:
print("Unique values of SECONDS in the time column:", data["Timestamp"].dt.second.unique())
print("Since the frequency is 1 min, we can safely floor the time to the nearest minute. But the trade itself will approximately happen in the next minute.")
data["Timestamp"] = data["Timestamp"].dt.floor(freq = "min")

Unique values of SECONDS in the time column: [59]
Since the frequency is 1 min, we can safely floor the time to the nearest minute. But the trade itself will approximately happen in the next minute.


In [8]:
# Trading occurs only on the first full week of a month
data["day"] = data["Timestamp"].dt.day_of_week
data["month"] = data["Timestamp"].dt.month
data["year"] = data["Timestamp"].dt.year

first_monday = data.loc[data["day"] == 0].groupby(["year", "month"])["Date"].min()

week_one = [pd.date_range(start = x, periods=5) for x in first_monday]
week_one_flat = [date for sublist in week_one for date in sublist]

data["IsWeek1"] = data["Date"].isin(week_one_flat).astype("float")
data["IsExpiry"] = np.where(data["day"] == 2, 1, 0)

trading_days = data.loc[data["IsWeek1"] == 1].copy()
print("Total no. of trading days = ", data.drop_duplicates("Date").IsWeek1.sum())

Total no. of trading days =  59.0


In [ ]:
trading_days["close_delta"] = abs(trading_days["Close"] - 50)

entry_mask = ((trading_days["Timestamp"].dt.hour == 9) & (trading_days["Timestamp"].dt.minute == 20))
entry = trading_days.loc[entry_mask].copy()

trading_days["close_delta_min"] = trading_days.groupby("Timestamp")["close_delta"].transform("min")

for option in ["CE, PE"]:
    trading_days.loc[(trading_days["Call/Put"] == option) & (trading_days["close_delta"] == trading_days["close_delta_min"])]

522         0.05
523         0.05
524         0.05
525         0.05
526         0.05
            ... 
10219964    4.50
10219965    4.50
10219966    4.50
10219967    4.50
10219968    4.50
Name: close_delta, Length: 6805, dtype: float64

In [ ]:
trading_days["close_delta_min"] = trading_days.groupby("Timestamp")["close_delta"].transform("min")

trading_days.loc[(trading_days["Call/Put"] == "CE") & (trading_days["close_delta"] == trading_days["close_delta_min"]) & entry_mask]

,Date,Ticker,Time,Open,High,Low,Close,Call/Put,Timestamp,Strike,UnderlyingClose,day,month,year,IsWeek1,IsExpiry,close_delta,close_delta_direction,close_delta_min
556,2023-01-02,BANKNIFTY44000CE,09:20:59,50.80,51.70,44.35,50.05,CE,2023-01-02 09:20:00,44000.0,42900.0,0,1,2023,1.0,0,0.05,1,0.05
113471,2023-01-05,BANKNIFTY43300CE,09:20:59,64.60,67.30,58.80,58.80,CE,2023-01-05 09:20:00,43300.0,42900.0,3,1,2023,1.0,0,8.80,1,8.80
152328,2023-01-06,BANKNIFTY43600CE,09:20:59,54.10,55.20,51.65,52.85,CE,2023-01-06 09:20:00,43600.0,42750.0,4,1,2023,1.0,0,2.85,1,2.85
1005865,2023-02-07,BANKNIFTY42400CE,09:20:59,49.95,51.60,47.60,48.60,CE,2023-02-07 09:20:00,42400.0,41400.0,1,2,2023,1.0,0,1.40,-1,1.40
1087505,2023-02-09,BANKNIFTY41800CE,09:20:59,48.85,52.05,46.85,48.60,CE,2023-02-09 09:20:00,41800.0,41300.0,3,2,2023,1.0,0,1.40,-1,1.40
1127836,2023-02-10,BANKNIFTY42400CE,09:20:59,53.35,53.35,50.10,50.90,CE,2023-02-10 09:20:00,42400.0,41400.0,4,2,2023,1.0,0,0.90,1,0.90
1790556,2023-03-06,BANKNIFTY42100CE,09:20:59,48.70,55.90,48.50,53.65,CE,2023-03-06 09:20:00,42100.0,40900.0,0,3,2023,1.0,0,3.65,1,3.65
2524890,2023-04-03,BANKNIFTY41300CE,09:20:59,60.15,60.15,53.60,53.60,CE,2023-04-03 09:20:00,41300.0,40400.0,0,4,2023,1.0,0,3.60,1,3.60
2603199,2023-04-06,BANKNIFTY41100CE,09:20:59,51.60,53.85,47.30,52.20,CE,2023-04-06 09:20:00,41100.0,40500.0,3,4,2023,1.0,0,2.20,1,2.20
3469109,2023-05-10,BANKNIFTY43700CE,09:20:59,51.45,52.90,48.80,50.85,CE,2023-05-10 09:20:00,43700.0,42750.0,2,5,2023,1.0,1,0.85,1,0.85


In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

In [2]:
ENTRY_TIME = pd.to_datetime("09:20", format = "%H:%M").time()
EXIT_TIME = pd.to_datetime("15:20", format = "%H:%M").time()

# class TradeSheet:
#     def __init__(self) -> None:
#         self.rows = []
    
#     def log_trade(self, data):
#         for row in data:
#             self.rows.append({
#                 "Entry Date": row["Date"],
#                 "Exit Date": row["Date"],
#                 "Entry Time": row["entry_time"],
#                 "Exit Time": row["time"],

#                 "Ticker": row["Ticker"],
#                 "Strike Price": row["Strike"],
#                 "Option Type": row["Call/Put"],
#                 "Entry Price": row["position_active_stop_loss"]/1.5,
#                 "Exit Price": row["position_active_stop_loss"],
#                 "Quantity": 15,
#                 "Entry Value": 15 * row["position_active_stop_loss"]/1.5,
#                 "Exit Value": 15 * row["position_active_stop_loss"],
                
#             })

# import pandas as pd

class TradeSheet:
    def __init__(self, lot_size=15):
        self.rows = []
        self.lot_size = lot_size

    def _to_row_dicts(self, data):
        # Normalize input to a list of dicts
        if isinstance(data, pd.DataFrame):
            return data.to_dict('records')
        if isinstance(data, dict):
            return [data]
        # list/iterable
        if len(data) == 0:
            return []
        if isinstance(data[0], dict):
            return data
        # e.g., list of Series/tuples -> coerce
        return pd.DataFrame(data).to_dict('records')

    def log_trade(self, data):
        rows = self._to_row_dicts(data)
        for row in rows:
            # graceful key fallbacks
            date_val   = row.get('Date') or row.get('date')
            etime      = row.get('entry_time') or row.get('Entry Time')
            xtime      = row.get('time') or row.get('exit_time') or row.get('Exit Time')
            ticker     = row.get('Ticker') or row.get('ticker')
            strike     = row.get('Strike') or row.get('strike') or row.get('Strike Price')
            opt_type   = row.get('Call/Put') or row.get('option_type') or row.get('Option Type')

            # entry/exit price logic:
            # If SL row provided with position_active_stop_loss, infer entry = stop/1.5 unless explicit entry/exit given
            entry_price = row.get('entry_price')
            exit_price  = row.get('exit_price')
            if entry_price is None and exit_price is None and 'position_active_stop_loss' in row:
                exit_price  = float(row['position_active_stop_loss'])
                entry_price = exit_price / 1.5

            if entry_price is None or exit_price is None:
                raise KeyError(f"Missing entry/exit price in row: {row}")

            qty = int(row.get('quantity', self.lot_size))
            entry_val = qty * entry_price
            exit_val  = qty * exit_price
            pnl       = exit_val - entry_val

            self.rows.append({
                "Entry Date": date_val,
                "Exit Date": date_val,         # adjust if your row carries a separate exit date
                "Entry Time": etime,
                "Exit Time": xtime,
                "Ticker": ticker,
                "Strike Price": strike,
                "Option Type": opt_type,       # "CE"/"PE"
                "Entry Price": entry_price,
                "Exit Price": exit_price,
                "Quantity": qty,
                "Entry Value": entry_val,
                "Exit Value": exit_val,
                "Gross P&L": pnl,
            })

    def to_dataframe(self):
        return pd.DataFrame(self.rows)


TRADE_SHEET = TradeSheet()

In [ ]:
def data_preprocessing(data):
    data["Time"] = pd.to_datetime(data["Time"], format = "%H:%M:%S")
    data["Date"] = pd.to_datetime(data["Date"], format = "%Y-%m-%d").dt.normalize()

    data['Strike'] = data['Ticker'].str.extract(r"(\d+)").astype("float")
    data["UnderlyingClose"] = data.groupby(["Date", "Time"])["Strike"].transform("median")

    data["day"] = data["Date"].dt.day_of_week
    data["month"] = data["Date"].dt.month
    data["year"] = data["Date"].dt.year

    first_monday = data.loc[data["day"] == 0].groupby(["year", "month"])["Date"].min()

    week_one = [pd.date_range(start = x, periods=5) for x in first_monday]
    week_one_flat = [date for sublist in week_one for date in sublist]

    data["is_week_one"] = data["Date"].isin(week_one_flat).astype("float")
    data["is_expiry"] = np.where(data["day"] == 2, 1, 0)

    trading_cal = data.loc[data["is_week_one"] == 1].copy()
    trading_cal["close_premium_diff"] = abs(trading_cal["Close"] - 50)
    trading_cal["close_premium_diff_min"] = trading_cal.groupby(["Date", "Time"])["close_premium_diff"].transform("min")
    
    trading_cal["position_active"] = False
    trading_cal["stop_loss_activated"] = False
    trading_cal["position_active_stop_loss"] = -1.0
    trading_cal["entry_time"] = datetime.time(0, 0, 0)

    return trading_cal
    

In [17]:
def strike_selection(data, time)->None:
    time_mask = ((data["Time"].dt.hour == time.hour) & (data["Time"].dt.minute == time.minute))
    entry = data.loc[time_mask & data["position_active"] == False]

    for leg_side, strike_asc in [("CE", True), ("PE", False)]:
        
        # in case of a tie in min(close_premium_diff) value:
        # for a CE, we want to choose the option with higher strike price - lower risk for the Seller, direction towards OTM
        # for a PE, we want to choose the option with lower strike price - lower risk for the Seller, direction towards OTM
        picks = (entry.loc[entry["Call/Put"] == leg_side]
                 .sort_values(["Date", "Time", "close_premium_diff_min", "Strike"],
                              ascending = [True, True, True, strike_asc]))
        
        idx = picks.groupby(["Date","Time"]).head(1).index
        if len(idx) == 0:
            continue
        # shorting the positions
        entry_price = data.loc[idx, "Close"]
        stop_level  = entry_price * 1.5

        data.loc[idx, "position_active"] = True
        data.loc[idx, "entry_time"] = time
        data.loc[idx, "position_active_stop_loss"] = stop_level
        data.loc[idx, "stop_loss_activated"] = False
        
    return


In [18]:
def exit_strategy(data, time)->None:
    time_mask = ((data["Time"].dt.hour == time.hour) & (data["Time"].dt.minute == time.minute))
    entered = data.loc[time_mask & data["position_active"] == True]

    if len(entered) == 0:
        return
    
    if time == EXIT_TIME:
        to_log = entered.copy()
        to_log["entry_price"] = (to_log["position_active_stop_loss"] / 1.5)
        to_log["exit_price"]  = to_log["Close"]
        TRADE_SHEET.log_trade(to_log)

        end_idx = entered.index
        data.loc[end_idx, "position_active"] = False
        data.loc[end_idx, "position_active_stop_loss"] = -1.0
        data.loc[end_idx, "entry_time"] = datetime.time(0, 0, 0)
        data.loc[end_idx, "stop_loss_activated"] = False
        return


    sl_activated_mask = entered["High"] >= entered["position_active_stop_loss"]
    if sl_activated_mask.any():
        hit = entered.loc[sl_activated_mask].copy()
        hit["entry_price"] = hit["position_active_stop_loss"] / 1.5
        hit["exit_price"]  = hit["position_active_stop_loss"]
        TRADE_SHEET.log_trade(entered.loc[hit])

        hit_idx = hit.index
        data.loc[hit_idx, "position_active"] = False
        data.loc[hit_idx, "position_active_stop_loss"] = -1
        data.loc[hit_idx, "entry_time"] = datetime.time(0, 0, 0)

    strike_selection(data, time)

    return


In [19]:
def signal_generation(data, entry_time=ENTRY_TIME, exit_time=EXIT_TIME):

    minutes = data["Time"].dt.time.unique()
    minutes = sorted([t for t in minutes if entry_time <= t <= exit_time])
    
    for minute in minutes:
        strike_selection(data, minute)
        exit_strategy(data, minute)
    
    return

In [7]:
data_og = pd.read_csv(f"../data/Options_data_2023.csv", index_col = 0)
data = data_og.copy()
data = data_preprocessing(data)

In [20]:
signal_generation(data)

KeyboardInterrupt: 

In [44]:
data["Time"].dt.time[0]

datetime.time(9, 15, 59)

""
